# NIF Deep Learning (NIFDL) for Classification Tasks
This notebook demonstrates the implementation of a quantum-classical hybrid model called NIF Deep Learning (NIFDL) using the Matchcake library. The model is designed for classification tasks and leverages the Non-Interacting Fermionic Device (NIFD) for quantum computations. The notebook also showcases the use of PyTorch Lightning for training and evaluation, as well as the Ax library for hyperparameter optimization.

In [9]:
import os
import datetime
import time
from pathlib import Path
from typing import Optional, Any
import json
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pennylane as qml
import torch
from ax import RangeParameterConfig
from matchcake import NonInteractingFermionicDevice
from matchcake.operations import SptmAngleEmbedding, CompRxRx, CompHH

from matchcake_opt.datamodules.datamodule import DataModule
from matchcake_opt.modules.classification_model import ClassificationModel
from matchcake_opt.tr_pipeline.automl_pipeline import AutoMLPipeline
from matchcake_opt.tr_pipeline.lightning_pipeline import LightningPipeline

In the following cell we define the NIFDL model class, which inherits from `ClassificationModel`. The model consists of a quantum circuit implemented using PennyLane, and it includes methods for building the model, defining the quantum circuit, and performing forward passes.

In [10]:
class NIFDL(ClassificationModel):
    MODEL_NAME = "NIFDL"
    DEFAULT_N_QUBITS = 16
    DEFAULT_LEARNING_RATE = 2e-4
    DEFAULT_N_LAYERS = 6

    HP_CONFIGS = [
        RangeParameterConfig(
            name="learning_rate",
            parameter_type="float",
            bounds=(1e-5, 0.1),
        ),
        RangeParameterConfig(
            name="n_qubits",
            parameter_type="int",
            bounds=(4, 32),
            step_size=2,
        ),
        RangeParameterConfig(
            name="n_layers",
            parameter_type="int",
            bounds=(1, 10),
        ),
    ]

    def __init__(
            self,
            input_shape: Optional[tuple[int, ...]],
            output_shape: Optional[tuple[int, ...]],
            learning_rate: float = DEFAULT_LEARNING_RATE,
            n_qubits: int = DEFAULT_N_QUBITS,
            n_layers: int = DEFAULT_N_LAYERS,
            **kwargs,
    ):
        super().__init__(input_shape=input_shape, output_shape=output_shape, learning_rate=learning_rate, **kwargs)
        self.save_hyperparameters("learning_rate", "n_qubits", "n_layers")
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.n_encoders = 8
        self.R_DTYPE = torch.float32
        self.C_DTYPE = torch.cfloat
        self.q_device = NonInteractingFermionicDevice(
            wires=self.n_qubits, r_dtype=self.R_DTYPE, c_dtype=self.C_DTYPE, show_progress=False
        )
        self.q_node = qml.QNode(self.circuit, self.q_device, interface="torch", diff_method="backprop")
        self._weight_shapes = {"weights": (self.n_layers, (self.n_qubits - 1) * 2)}
        self.flatten = torch.nn.Flatten()
        self.encoders = torch.nn.ModuleList(
            [
                qml.qnn.TorchLayer(self.q_node, self._weight_shapes)
                for _ in range(self.n_encoders)
            ]
        )
        self.readout = torch.nn.LazyLinear(self.output_size)
        self._build()

    def _build(self):
        dummy_input = torch.randn((3, *self.input_shape)).to(device=self.device)
        with torch.no_grad():
            self(dummy_input)
        return self

    def circuit(self, inputs, weights):
        SptmAngleEmbedding(inputs, wires=range(self.n_qubits))
        for i in range(self.n_layers):
          for j in range(self.n_qubits - 1):
            CompRxRx(weights[i, j*2 : j*2+2], wires=[j, j+1])
            CompHH(wires=[j, j+1])
        return [qml.expval(qml.PauliZ(wires=i)) for i in range(self.n_qubits)]

    def forward(self, x) -> Any:
        x = self.flatten(x).to(self.device)
        x_split = torch.split(x, self.n_qubits, dim=1)
        x_out = [layer(x_chunk) for layer, x_chunk in zip(self.encoders, x_split)]
        x = torch.cat(x_out, dim=1).to(self.device)
        x = self.readout(x)
        return x

    @property
    def input_size(self):
        return int(np.prod(self.input_shape))

    @property
    def output_size(self):
        return int(np.prod(self.output_shape))

In [11]:
# Dataset
dataset_name = "Digits2D"
fold_id = 0
batch_size = 32
random_state = 0
num_workers = 0

# Model
model_cls = NIFDL

# Pipeline
job_output_folder = Path(os.getcwd()) / "data" / "automl" / dataset_name / model_cls.MODEL_NAME
checkpoint_folder = Path(job_output_folder) / "checkpoints"
pipeline_args = dict(
    max_epochs=128,  # increase at least to 256
    max_time="00:00:01:00",  # DD:HH:MM:SS, increase at least to "00:01:00:00"
)

Then, we set up the data module using the specified dataset and parameters.

In [12]:
datamodule = DataModule.from_dataset_name(
    dataset_name,
    fold_id=fold_id,
    batch_size=batch_size,
    random_state=random_state,
    num_workers=num_workers,
)

## Lightning Pipeline - Model Training and Evaluation
In this section, we create a Lightning Pipeline for training and evaluating the NIFDL model.

In [13]:
model_args = dict(
    n_qubits=16,
    learning_rate=2e-4,
    n_layers=6,
)
lightning_pipeline = LightningPipeline(
    model_cls=model_cls,
    datamodule=datamodule,
    checkpoint_folder=checkpoint_folder,
    max_epochs=10,
    max_time="00:00:01:00",  # DD:HH:MM:SS
    overwrite_fit=True,
    verbose=True,
    accelerator="cpu",
    **model_args,
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [14]:
start_time = time.perf_counter()
metrics = lightning_pipeline.run()
end_time = time.perf_counter()
elapsed_time = datetime.timedelta(seconds=end_time - start_time)
print(f"Time taken: {elapsed_time}")
print("⚡" * 20, "\nValidation Metrics:\n", metrics, "\n", "⚡" * 20)
test_metrics = lightning_pipeline.run_test()
print("⚡" * 20, "\nTest Metrics:\n", test_metrics, "\n", "⚡" * 20)


  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | val_loss      | NLLLoss          | 0      | train
1 | train_loss    | NLLLoss          | 0      | train
2 | _metrics      | MetricCollection | 0      | train
3 | train_metrics | MetricCollection | 0      | train
4 | val_metrics   | MetricCollection | 0      | train
5 | test_metrics  | MetricCollection | 0      | train
6 | flatten       | Flatten          | 0      | train
7 | encoders      | ModuleList       | 1.4 K  | train
8 | readout       | Linear           | 650    | train
-----------------------------------------------------------
2.1 K     Trainable params
0         Non-trainable params
2.1 K     Total params
0.008     Total estimated model params size (MB)
37        Modules in train mode
0         Modules in eval mode


Checkpoint folder: E:\Github\MatchCake-Opt\notebooks\data\automl\Digits2D\NIFDL\checkpoints
Model: NIFDL
NIFDL(
  (val_loss): NLLLoss()
  (train_loss): NLLLoss()
  (_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassF1Score): MulticlassF1Score()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassAUROC): MulticlassAUROC(),
    prefix=train_
  )
  (train_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassF1Score): MulticlassF1Score()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassAUROC): MulticlassAUROC(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassF1Score): MulticlassF1Score()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassAUROC): MulticlassAUROC(),
    prefix=v

Time limit reached. Elapsed time is 0:01:00. Signaling Trainer to stop.



Epoch Progress:  10%|█         | 1/10 [01:21<12:11, 81.23s/it, v_num=0, train_loss=2.32, val_loss=2.3, val_MulticlassAccuracy=0.0988, val_MulticlassF1Score=0.0988, val_MulticlassRecall=0.0988, val_MulticlassPrecision=0.0988, val_MulticlassAUROC=0.586]

Restoring states from the checkpoint path at E:\Github\MatchCake-Opt\notebooks\data\automl\Digits2D\NIFDL\checkpoints\epoch00-step0019.ckpt
Loaded model weights from the checkpoint at E:\Github\MatchCake-Opt\notebooks\data\automl\Digits2D\NIFDL\checkpoints\epoch00-step0019.ckpt


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    val_MulticlassAUROC    │    0.5861977338790894     │
│  val_MulticlassAccuracy   │    0.09876543283462524    │
│   val_MulticlassF1Score   │    0.09876543283462524    │
│  val_MulticlassPrecision  │    0.09876543283462524    │
│   val_MulticlassRecall    │    0.09876543283462524    │
│         val_loss          │    2.3029329776763916     │
└───────────────────────────┴───────────────────────────┘

Restoring states from the checkpoint path at E:\Github\MatchCake-Opt\notebooks\data\automl\Digits2D\NIFDL\checkpoints\epoch00-step0019.ckpt
Loaded model weights from the checkpoint at E:\Github\MatchCake-Opt\notebooks\data\automl\Digits2D\NIFDL\checkpoints\epoch00-step0019.ckpt


Metrics saved to E:\Github\MatchCake-Opt\notebooks\data\automl\Digits2D\NIFDL\checkpoints\validation_metrics.json
Time taken: 0:01:42.762405
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Validation Metrics:
 {'val_loss': 2.3029329776763916, 'val_MulticlassAccuracy': 0.09876543283462524, 'val_MulticlassF1Score': 0.09876543283462524, 'val_MulticlassRecall': 0.09876543283462524, 'val_MulticlassPrecision': 0.09876543283462524, 'val_MulticlassAUROC': 0.5861977338790894, 'validation_time': 17.814280099992175, 'training_time': 84.94243890000507} 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   test_MulticlassAUROC    │     0.579858660697937     │
│  test_MulticlassAccuracy  │    0.10555555671453476    │
│  test_MulticlassF1Score   │    0.10555555671453476    │
│ test_MulticlassPrecision  │    0.10555555671453476    │
│   test_MulticlassRecall   │    0.10555555671453476    │
│         test_loss         │     2.303359031677246     │
└───────────────────────────┴───────────────────────────┘

Metrics saved to E:\Github\MatchCake-Opt\notebooks\data\automl\Digits2D\NIFDL\checkpoints\test_metrics.json
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Test Metrics:
 {'test_loss': 2.303359031677246, 'test_MulticlassAccuracy': 0.10555555671453476, 'test_MulticlassF1Score': 0.10555555671453476, 'test_MulticlassRecall': 0.10555555671453476, 'test_MulticlassPrecision': 0.10555555671453476, 'test_MulticlassAUROC': 0.579858660697937, 'test_time': 9.541893699992215} 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡


# AutoML Pipeline - Hyperparameter Optimization

In [15]:
automl_pipeline = AutoMLPipeline(
    model_cls=model_cls,
    datamodule=datamodule,
    checkpoint_folder=checkpoint_folder,
    automl_iterations=5,  # increase at least to 32
    inner_max_epochs=10,  # increase at least to 128
    inner_max_time="00:00:01:00",  # increase at least to "00:00:10:00"
    automl_overwrite_fit=True,
    accelerator="cpu",
    **pipeline_args
)

In [ ]:
start_time = time.perf_counter()
automl_pipeline.run()
end_time = time.perf_counter()
print(f"Time taken: {end_time - start_time:.4f} seconds")


AutoML iterations:   0%|          | 0/5 [00:00<?, ?it/s][INFO 11-28 11:34:11] ax.api.client: GenerationStrategy(name='Center+Sobol+MBM:fast', nodes=[CenterGenerationNode(next_node_name='Sobol'), GenerationNode(node_name='Sobol', generator_specs=[GeneratorSpec(generator_enum=Sobol, model_key_override=None)], transition_criteria=[MinTrials(transition_to='MBM'), MinTrials(transition_to='MBM')]), GenerationNode(node_name='MBM', generator_specs=[GeneratorSpec(generator_enum=BoTorch, model_key_override=None)], transition_criteria=[])]) chosen based on user input and problem structure.
[INFO 11-28 11:34:11] ax.api.client: Generated new trial 0 with parameters {'learning_rate': 0.050005, 'n_qubits': 18, 'n_layers': 5} using GenerationNode CenterOfSearchSpace.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
print(f"Best Hyperparameters:\n{json.dumps(automl_pipeline.get_best_params(), indent=2, default=str)}")

In [ ]:
lt_pipeline, metrics = automl_pipeline.run_best_pipeline()
print("⚡" * 20, "\nValidation Metrics:\n", metrics, "\n", "⚡" * 20)

In [ ]:
test_metrics = lt_pipeline.run_test()
print("⚡" * 20, "\nTest Metrics:\n", test_metrics, "\n", "⚡" * 20)

-------------------------------------------------------